In [31]:
import numpy as np


# We need an algorithm to perform belief propagation on our hmm
def forward_HMM(A, B, pi, observed):
    """
    A: transition
    B: emission
    pi: initial
    n_nodes: number of nodes in the chain
    observed: list containing observed ones.
    """
    n_nodes = len(observed)
    n_states = A.shape[0]
    alpha = np.zeros((n_nodes, n_states))

    for j in range(n_states):
        alpha[0, j] = pi[j] * B[j, observed[0]]

    for i in range(1, n_nodes):
        for j in range(n_states):
            for k in range(n_states):
                alpha[i, j] = (
                    alpha[i, j] + A[k, j] * B[j, observed[i]] * alpha[i - 1, k]
                )

    return alpha

In [32]:
# We need an algorithm to perform belief propagation on our hmm
def backward_HMM(A, B, observed):
    """
    A: transition
    B: emission
    n_nodes: number of nodes in the chain
    observed: list containing observed ones.
    """
    n_nodes = len(observed)
    n_states = A.shape[0]
    beta = np.zeros((n_nodes - 1, n_states))

    for j in range(n_states):
        for k in range(n_states):
            beta[-1, j] = beta[-1, j] + A[j, k] * B[k, observed[n_nodes - 1]]

    for i in range(n_nodes - 3, -1, -1):
        for j in range(n_states):
            for k in range(n_states):
                beta[i, j] = (
                    beta[i, j] + A[j, k] * B[k, observed[i + 1]] * beta[i + 1, k]
                )

    return beta

In [7]:
A = np.array([[0.6, 0.4], [0.3, 0.7]])
B = np.array([[0.5, 0.5], [0.1, 0.9]])
pi = np.array([0.2, 0.8])
observed = np.array([1, 0, 1])

In [8]:
beta = backward_HMM(A, B, observed)

In [9]:
alpha = forward_HMM(A, B, pi, observed)

In [10]:
def compute_conditional(alpha, beta, i):
    """
    alpha: list containing forward messages
    beta: list containing backward messages
    i : hidden element for which you want the conditional on the observed variables (i = 1, ..., M)
    """
    if i == 0:
        raise ValueError("no zio serve il numero di variabile")

    if i == alpha.shape[0]:
        return alpha[i - 1] / np.sum(alpha[i - 1])

    gamma = alpha[i - 1] * beta[i - 1]
    gamma = gamma / np.sum(gamma)

    return gamma

In [11]:
def compute_all_conditional(alpha, beta):
    """
    alpha: list containing forward messages
    beta: list containing backward messages
    """
    n_nodes = alpha.shape[0]
    n_states = alpha.shape[1]

    gamma = np.zeros((n_nodes, n_states))

    gamma[n_nodes - 1] = alpha[n_nodes - 1] / np.sum(alpha[n_nodes - 1])

    for i in range(n_nodes - 1):
        gamma[i] = alpha[i] * beta[i] / np.sum(alpha[i] * beta[i])

    return gamma

In [12]:
alpha = forward_HMM(A, B, pi, observed)
beta = backward_HMM(A, B, observed)
gamma = compute_all_conditional(alpha, beta)

In [23]:
def divide_row_by_sum(matrix):
    row_sums = np.sum(matrix, axis=1)  # Calculate the sum of each row
    divided_matrix = (
        matrix / row_sums[:, np.newaxis]
    )  # Divide each element by the corresponding row sum
    return divided_matrix

In [24]:
def update_B(gamma, observed):
    # n_nodes = gamma.shape[0]
    n_states = gamma.shape[1]

    B = np.zeros((n_states, n_states))

    for i in range(n_states):
        for j in range(n_states):
            for k in range(len(observed)):
                if observed[k] == j:
                    B[i, j] += gamma[k, i]

    return divide_row_by_sum(B)

In [25]:
update_B(gamma, observed)

array([[0.55685987, 0.44314013],
       [0.17905611, 0.82094389]])

In [26]:
def Baum_Welch(A, B_start, pi, observed, maxIter=100):
    B = np.copy(B_start)
    for it in range(maxIter):
        alpha = forward_HMM(A, B, pi, observed)
        beta = backward_HMM(A, B, observed)
        gamma = compute_all_conditional(alpha, beta)
        B = update_B(gamma, observed)
    return B

In [27]:
A = np.array([[0.6, 0.4], [0.3, 0.7]])
pi = np.array([0.2, 0.8])
observed = np.array([1, 0, 1])

In [28]:
B_start = np.zeros((2, 2)) + 0.5
Baum_Welch(A, B_start, pi, observed)

array([[0.44294088, 0.55705912],
       [0.27897937, 0.72102063]])

In [ ]:
import numpy as np


def baum_welch(observed_sequence, num_states, num_emissions, num_iterations):
    # Initialize emission probabilities randomly
    # emission_probs = np.random.rand(num_states, num_emissions)
    emission_probs = np.full((num_states, num_emissions), 1 / 27)
    emission_probs /= np.sum(emission_probs, axis=1, keepdims=True)

    # Initialize the observed sequence and its length
    observed_sequence = np.array(observed_sequence)
    T = len(observed_sequence)

    for iteration in range(num_iterations):
        # Step 1: Forward-Backward Algorithm

        # Forward variables
        alpha = np.zeros((T, num_states))

        # Calculate alpha(1, i)
        alpha[0] = emission_probs[:, observed_sequence[0]]

        # Calculate alpha(t, i) for t > 1
        for t in range(1, T):
            for i in range(num_states):
                alpha[t, i] = (
                    np.sum(alpha[t - 1] * transition_probs[:, i])
                    * emission_probs[i, observed_sequence[t]]
                )

        # Backward variables
        beta = np.zeros((T, num_states))

        # Set beta(T, i) = 1
        beta[T - 1] = 1

        # Calculate beta(t, i) for t < T
        for t in range(T - 2, -1, -1):
            for i in range(num_states):
                beta[t, i] = np.sum(
                    beta[t + 1]
                    * transition_probs[i, :]
                    * emission_probs[:, observed_sequence[t + 1]]
                )

        # Step 2: Estimation Step

        # Compute gamma variables
        gamma = alpha * beta / np.sum(alpha * beta, axis=1, keepdims=True)

        # Compute xi variables
        xi = np.zeros((T - 1, num_states, num_states))
        for t in range(T - 1):
            for i in range(num_states):
                for j in range(num_states):
                    xi[t, i, j] = (
                        alpha[t, i]
                        * transition_probs[i, j]
                        * emission_probs[j, observed_sequence[t + 1]]
                        * beta[t + 1, j]
                    ) / np.sum(
                        alpha[t, :]
                        * transition_probs[:, j]
                        * emission_probs[j, observed_sequence[t + 1]]
                        * beta[t + 1, j]
                    )

        # Step 3: Maximization Step

        # Update emission probabilities
        for i in range(num_states):
            for k in range(num_emissions):
                emission_probs[i, k] = np.sum(
                    gamma[:, i] * (observed_sequence == k)
                ) / np.sum(gamma[:, i])

    return emission_probs